In [4]:
# 1. 라이브러리 가져오기
import pymysql

# 2. 접속하기
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

# 3. 커서 가져오기
cursor = db.cursor()

sql = """
    create table bookstore_best (
        title varchar(200) not null,
        ranks varchar(200),
        author varchar(20),
        publs varchar(20),
        publ_date varchar(20),
        e_book varchar(20),
        low_price varchar(20),
        price varchar(20),        
    );
"""

# 5. SQL 구문 실행하기
cursor.execute(sql)

# 6. DB에 Complete 하기
db.commit()

# 7. DB 연결 닫기
db.close()

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')' at line 10")

In [6]:
import requests
import time
from selenium import webdriver

import pymysql
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36')
from bs4 import BeautifulSoup


driver = webdriver.Chrome(options=options)
url = "https://search.shopping.naver.com/book/search/category?bookTabType=BEST_SELLER&catId=50005542&pageIndex=1&pageSize=40"
driver.get(url)
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
res = requests.get(url, headers) 
res.raise_for_status()
        
#스크롤    
prev_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)
    curr_height = driver.execute_script("return document.body.scrollHeight")

    if curr_height == prev_height:
        break

    prev_height = curr_height    

soup = BeautifulSoup(driver.page_source, "lxml")

titles = soup.find_all('span',{'class' : 'bookListItem_text__bglOw'}) #제목
#    genres = soup.find_all('div',{'class' : 'bookListItem_feature__txTlp'}) #장르. 순위
genres_con = soup.find_all('div',{'class' : 'bookListItem_text_area__hF892'}) #버전2 장르
authors = soup.find_all('span',{'class' : 'bookListItem_author__fdjcn'}) #작가
publs = soup.select('span.bookListItem_detail_publish__FgPYQ') #출판사
ebooks = soup.select('div.bookListItem_sub_info__AfkOO') #도서,e북, 가격

for title,genre_con,author, publ, ebook in zip(titles,genres_con, authors, publs, ebooks):

    #제목, 순위
    title = title.get_text()
    print(title)

    #작가
    a,b = author.get_text().split('작가')
    print("작가 :" +b)

     #출판
    pub = publ.get_text()
    if pub.find('20') != -1:
        pub_sa= pub[:-11].lstrip("출판") #출판사
        pub_date=pub[-11:] #날짜
        print(pub_sa)
        print(pub_date)
    else:
        pub_sa = pub.lstrip("출판") #출간날짜가 없을때 출판사
        pub_date = "출간날짜 없음"
        print(pub_sa) 
        print(pub_date)        

    # 도서, e북 : 판매처
    eboo= ebook.get_text()
    if eboo.find('도서' and 'e북') != -1:
        book_ebook = "도서 & e북"
        print(book_ebook)
    elif eboo.find('도서') != -1:
        book_ebook = "도서"
        print(book_ebook)
    elif eboo.find('e북') != -1:
        book_ebook = 'e북'
        print(book_ebook)
    else:
        book_ebook = '판매없음'
        print(book_ebook) 
    #판매가격
    ebo= ebook.get_text()
    a= ebo.find('최저')
    b= ebo.find('구매')
    c= ebo.find('신고')
    e= ebo.find('e북')
    if ebo.find('최저'and '구매') != -1:
        book_price = ebo[a+3:e] 
        e_price = ebo[b+3:c]
        print("도서최저 : " + book_price) #최저
        print("e북구매 : " +  e_price) #구매
        print()

    else :
        book_price = ebo[a+3:c].rstrip("원")
        e_price = "ebook판매 안함"
        print("도서최저 : " + book_price) #최저
        print(e_price)
        print()
    sql = """INSERT INTO bookstore_best VALUES(
    "{0}", "{1}", "{2}", "{3}", "{4}", "{5}", "{6}", "{7}");""".format(title, title, b, pub_sa, pub_date, book_ebook, book_price, e_price)
    cursor.execute(sql)
    
db.commit()
cursor.close()
db.close()


1위하얼빈 
작가 :김훈
문학동네
2022.08.03.
도서 & e북
도서최저 : 14,400원
e북구매 : 10,080원

2위파친코 2 
작가 :이민진
인플루엔셜(주)
2022.08.25.
도서
도서최저 : 14,220
ebook판매 안함

3위역행자 
작가 :자청
웅진지식하우스
2022.05.30.
도서 & e북
도서최저 : 15,750원
e북구매 : 11,070원

4위불편한 편의점 2 
작가 :김호연
나무옆의자
2022.08.10.
도서
도서최저 : 12,600
ebook판매 안함

5위헤어질 결심 각본 
작가 :정서경, 박찬욱
을유문화사
2022.08.05.
도서 & e북
도서최저 : 13,500원
e북구매 : 9,450원

6위계속 가보겠습니다 
작가 :임은정
메디치미디어
2022.07.22.
도서 & e북
도서최저 : 15,390원
e북구매 : 11,340원

7위파친코 1 
작가 :이민진
인플루엔셜
2022.08.05.
도서
도서최저 : 14,220
ebook판매 안함

8위불편한 편의점 
작가 :김호연
나무옆의자
2021.04.20.
도서 & e북
도서최저 : 12,600원
e북구매 : 8,820원

9위이상한 변호사 우영우 1 
작가 :문지원
김영사
출간날짜 없음
도서
도서최저 : 17,820
ebook판매 안함

10위당신이라는 기적 
작가 :정한경
북로망스
2022.07.07.
도서 & e북
도서최저 : 14,400원
e북구매 : 10,080원

11위이상한 변호사 우영우 2 
작가 :문지원
김영사
출간날짜 없음
도서
도서최저 : 17,820
ebook판매 안함

12위흔한남매 11 
작가 :흔한남매 (원작), 백난도 외 1명
미래엔아이세움
2022.07.28.
도서
도서최저 : 13,050
ebook판매 안함

14위호감 가는 말투에는 비밀이 있다 
작가 :장신웨
리드리드출판
2022.08.11.
도서 & e북
도서최저 : 15,120원
e북구매 : 10,590원

15위미래의 부를 위한 투자 공부 
작가 :신진상
미디어숲
2022.0

In [2]:
driver.close()